In [ ]:
%cd /content/drive/MyDrive/BIG_PROJ # 대규모 jsonl 받아오기

/content/drive/MyDrive/BIG_PROJ


In [ ]:
# ✅ 사전 설치
!pip install transformers datasets chromadb langchain langchain-openai sentence-transformers torch tensorflow tqdm langchain_chroma faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.5 MB/s eta 0:0

In [ ]:
access_token = #
ACCESS_TOKEN = #

OPENAI_API_KEY= #

# db생성

In [ ]:
# ✅ 라이브러리 임포트
import json
import os
import shutil
from tqdm import tqdm

from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate

from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np



In [ ]:
from tqdm import tqdm
import numpy as np

# 모든 파일에서 plain_text 수집
def collect_all_texts(jsonl_paths):
    all_texts = []
    for path in jsonl_paths:
        with open(path, "r", encoding="utf-8") as f:
            lines = f.readlines()
            for line in tqdm(lines, desc=f"📂 텍스트 수집 중: {os.path.basename(path)}"):
                data = json.loads(line)
                text = data["metadata"].get("plain_text")
                if text:
                    all_texts.append(text)
    return all_texts

# 토큰 길이 분포 확인
def show_token_length_distribution(texts, sample_size=100_000):
    import random
    sampled = random.sample(texts, k=min(sample_size, len(texts)))
    print(f"🧪 토큰 길이 분석 (샘플 {len(sampled):,}건)")
    lengths = [len(tokenizer.encode(t)) for t in tqdm(sampled, desc="🔍 토큰 길이 측정")]
    for p in [50, 75, 90, 95, 99, 99.5, 99.9]:
        print(f"{p:5.1f}퍼센트 이내: {np.percentile(lengths, p):.1f} tokens")
    print(f"🧵 최대 길이: {max(lengths)} tokens")
# 실행
jsonl_paths = [
    "장치-독립항-공개공보-청구항구조화정보.jsonl",
    "장치-종속항-공개공보-청구항구조화정보.jsonl"
]

texts = collect_all_texts(jsonl_paths)
show_token_length_distribution(texts)

📂 텍스트 수집 중: 장치-종속항-공개공보-청구항구조화정보.jsonl: 100%|██████████| 4789718/4789718 [00:26<00:00, 177670.92it/s]


🧪 토큰 길이 분석 (샘플 100,000건)


🔍 토큰 길이 측정: 100%|██████████| 100000/100000 [00:52<00:00, 1916.04it/s]

 50.0퍼센트 이내: 45.0 tokens
 75.0퍼센트 이내: 67.0 tokens
 90.0퍼센트 이내: 99.0 tokens
 95.0퍼센트 이내: 130.0 tokens
 99.0퍼센트 이내: 224.0 tokens
 99.5퍼센트 이내: 277.0 tokens
 99.9퍼센트 이내: 472.0 tokens
🧵 최대 길이: 2307 tokens


In [ ]:
import os, json, gc
import numpy as np
from tqdm import tqdm
import torch
from transformers import BertModel, BertTokenizer
import faiss

# 설정
MODEL_NAME = "KIPI-ai/KorPatElectra"
ACCESS_TOKEN = "#"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 및 토크나이저 로딩 (최적화)
model = BertModel.from_pretrained(MODEL_NAME, token=ACCESS_TOKEN).to(DEVICE).eval().half()
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME, token=ACCESS_TOKEN)

# JSONL 로드 함수 (tqdm 포함)
def load_jsonl(path):
    texts, metadatas = [], []
    with open(path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        for line in tqdm(lines, desc=f"📂 텍스트 수집 중: {os.path.basename(path)}", unit="줄"):
            data = json.loads(line)
            text = data["metadata"].get("plain_text")
            if text:
                texts.append(text)
                meta = data.get("metadata", {})
                meta["raw_text"] = data.get("text")
                metadatas.append(meta)
    return texts, metadatas

# 임베딩 함수 (256 토큰 최적 설정)
def embed_texts(texts, batch_size=4096, max_length=256):
    vectors = []
    for i in tqdm(range(0, len(texts), batch_size), desc="🔍 임베딩 중", unit="batch"):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding="max_length", truncation=True,
                           max_length=max_length, return_token_type_ids=False)
        inputs = {k: v.to(DEVICE, non_blocking=True) for k, v in inputs.items()}

        with torch.inference_mode():
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]
            cls_embeddings = torch.nn.functional.normalize(cls_embeddings, p=2, dim=1)
            vectors.append(cls_embeddings.cpu().numpy())

        torch.cuda.empty_cache()
        gc.collect()

    return np.vstack(vectors)

# FAISS 인덱스 생성 및 저장
def build_faiss_index(jsonl_paths, index_save_path, batch_size=4096, chunk_size=500_000, max_length=256):
    dim = 768
    index = faiss.IndexFlatIP(dim)
    metadata_store = []

    for path in jsonl_paths:
        texts, metas = load_jsonl(path)
        print(f"📦 {len(texts):,}개 로드됨: {path}")

        for i in range(0, len(texts), chunk_size):
            chunk_texts = texts[i:i+chunk_size]
            chunk_metas = metas[i:i+chunk_size]

            print(f"🔢 임베딩 중: {i}-{i+len(chunk_texts)}")
            chunk_vectors = embed_texts(chunk_texts, batch_size=batch_size, max_length=max_length)
            index.add(chunk_vectors)
            metadata_store.extend(chunk_metas)

            print(f"✅ {len(chunk_vectors):,}개 벡터 저장됨 (index size: {index.ntotal:,})")

    faiss.write_index(index, index_save_path)
    with open(index_save_path + ".meta.json", "w", encoding="utf-8") as f:
        json.dump(metadata_store, f, ensure_ascii=False)
    print(f"🎉 FAISS 인덱스 저장 완료: {index_save_path}")

    return index, metadata_store

# 검색 함수
def search_faiss(index, query, top_k=3):
    with torch.inference_mode():
        inputs = tokenizer(query, return_tensors="pt", padding="max_length", truncation=True,
                           max_length=256, return_token_type_ids=False).to(DEVICE)
        output = model(**inputs).last_hidden_state[:, 0, :]
        query_vec = torch.nn.functional.normalize(output, p=2, dim=1).cpu().numpy()
    scores, indices = index.search(query_vec, top_k)
    return scores[0], indices[0]

jsonl_paths = [
    "장치-독립항-공개공보-청구항구조화정보.jsonl",
    "장치-종속항-공개공보-청구항구조화정보.jsonl"
]

index_save_path = "/content/korpat_index.faiss"

# 벡터 생성 및 저장
index, metas = build_faiss_index(jsonl_paths, index_save_path)

scores, indices = search_faiss(index, "배터리 셀을 보호하는 절연 구조")
for rank, idx in enumerate(indices):
    print(f"\n🔹 Top {rank+1}")
    print("📄", metas[idx].get("plain_text"))
    print("📑", metas[idx].get("raw_text"))


You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
📂 텍스트 수집 중: 장치-독립항-공개공보-청구항구조화정보.jsonl: 100%|██████████| 1876515/1876515 [00:10<00:00, 182733.39줄/s]


📦 1,876,502개 로드됨: 장치-독립항-공개공보-청구항구조화정보.jsonl
🔢 임베딩 중: 0-500000


🔍 임베딩 중: 100%|██████████| 123/123 [07:13<00:00,  3.52s/batch]


✅ 500,000개 벡터 저장됨 (index size: 500,000)
🔢 임베딩 중: 500000-1000000


🔍 임베딩 중: 100%|██████████| 123/123 [06:42<00:00,  3.27s/batch]


...

✅ 500,000개 벡터 저장됨 (index size: 5,876,502)
🔢 임베딩 중: 4000000-4500000


🔍 임베딩 중: 100%|██████████| 123/123 [09:42<00:00,  4.73s/batch]


✅ 500,000개 벡터 저장됨 (index size: 6,376,502)
🔢 임베딩 중: 4500000-4789718


🔍 임베딩 중: 100%|██████████| 71/71 [05:46<00:00,  4.88s/batch]


✅ 289,718개 벡터 저장됨 (index size: 6,666,220)
🎉 FAISS 인덱스 저장 완료: /content/korpat_index.faiss

🔹 Top 1
📄 상기 배터리 셀을 보호하는 셀 보호 회로와
📑 <ANY>상기 배터리 셀</ANY><RELATION_321>을 보호하는</RELATION_321> <COMPONENT>셀 보호 회로</COMPONENT>와

🔹 Top 2
📄 전극 탭을 포함하는 배터리 셀
📑 <ANY>전극 탭</ANY><RELATION_3>을 포함하는</RELATION_3> <COMPONENT>배터리 셀</COMPONENT>

🔹 Top 3
📄 전극 탭을 포함하는 배터리 셀
📑 <ANY>전극 탭</ANY><RELATION_3>을 포함하는</RELATION_3> <COMPONENT>배터리 셀</COMPONENT>


## test

In [ ]:
import os
import json
import numpy as np
import torch
from transformers import BertModel, BertTokenizer
import faiss
from IPython.display import display, Markdown

# ✅ 설정
INDEX_PATH = "/content/drive/MyDrive/BIG_PROJ/korpat_index.faiss"
META_PATH = INDEX_PATH + ".meta.json"
MODEL_NAME = "KIPI-ai/KorPatElectra"
ACCESS_TOKEN = "#"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # ✅ 모델 및 토크나이저 로딩
model = BertModel.from_pretrained(MODEL_NAME, token=ACCESS_TOKEN).to(DEVICE).eval().half()
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME, token=ACCESS_TOKEN)

# # ✅ FAISS 인덱스 및 메타데이터 로딩
# index = faiss.read_index(INDEX_PATH)
# with open(META_PATH, "r", encoding="utf-8") as f:
#     metadata = json.load(f)

# print(f"🔗 인덱스 로딩 완료: {index.ntotal:,} vectors")


You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import json

# Load metadata if not already loaded
if 'metadata' not in locals() or not metadata:
    META_PATH = "/content/drive/MyDrive/BIG_PROJ/korpat_index.faiss.meta.json"
    try:
        with open(META_PATH, "r", encoding="utf-8") as f:
            metadata = json.load(f)
        print(f"🔗 메타데이터 로딩 완료: {len(metadata):,} entries")
    except FileNotFoundError:
        print(f"❌ 오류: 메타데이터 파일 '{META_PATH}'를 찾을 수 없습니다.")
        metadata = []

# Print the first 10 metadata entries without formatting
if metadata:
    print(metadata[:10])
else:
    print("\n⚠️ 메타데이터가 비어있거나 로드되지 않았습니다.")

[{'app_num': '1019400024294', 'claim_num': 1, 'is_independent': True, 'sent_num': 1, 'sent_total': 4, 'is_last': False, 'plain_text': '내부에 히터가 설치된 상,하부틀과', 'raw_text': '<ANY>내부에 히터</ANY><RELATION_117>가 설치된</RELATION_117> <COMPONENT>상,하부틀</COMPONENT>과'}, {'app_num': '1019400024294', 'claim_num': 1, 'is_independent': True, 'sent_num': 2, 'sent_total': 4, 'is_last': False, 'plain_text': '상,하부틀의 위치하는 단열재와, 상,하부틀과 단열재사이에 분할되게 삽입되어 단열재와 상,하부틀사이에 공간부를 형성하는 중간체와, 상기 단열제의 외측에 부착되는 판체로 구성된 적충체와', 'raw_text': '<ANY>상,하부틀의 위치하는 단열재와, 상,하부틀과 단열재사이에 분할되게 삽입되어 단열재와 상,하부틀사이에 공간부를 형성하는 중간체와, 상기 단열제의 외측</ANY><RELATION_396841>에 부착되는 판체로 구성된</RELATION_396841> <COMPONENT>적충체</COMPONENT>와'}, {'app_num': '1019400024294', 'claim_num': 1, 'is_independent': True, 'sent_num': 3, 'sent_total': 4, 'is_last': False, 'plain_text': '상기 적충체를 결착하기 위한 채결 수단과', 'raw_text': '<ANY>상기 적충체</ANY><RELATION_214069>를 결착하기 위한</RELATION_214069> <COMPONENT>채결 수단</COMPONENT>과'}, {'app_num': '1019400024294', 'claim_num': 1, 'is_indep

In [ ]:
from collections import defaultdict

def search_korpat(query, top_k=5):
    with torch.inference_mode():
        inputs = tokenizer(query, return_tensors="pt", padding="max_length", truncation=True,
                           max_length=256, return_token_type_ids=False).to(DEVICE)
        vec = model(**inputs).last_hidden_state[:, 0, :]
        vec = torch.nn.functional.normalize(vec, p=2, dim=1).cpu().numpy()

    scores, indices = index.search(vec, top_k)

    for i, idx in enumerate(indices[0]):
        meta = metadata[idx]
        app_no = meta["app_num"]
        claim_no = meta["claim_num"]
        is_indep = meta["is_independent"]

        # 청구항에 속한 모든 문장 추출 및 정렬
        claim_sentences = sorted(
            [m for m in metadata if m["app_num"] == app_no and m["claim_num"] == claim_no],
            key=lambda x: x["sent_num"]
        )
        full_claim_text = "".join([m["plain_text"] for m in claim_sentences])

        print(f"\n🔹 Top {i+1} (score: {scores[0][i]:.4f})")
        print(f"🆔 특허번호: {app_no} | 청구항: {claim_no} ({'독립항' if is_indep else '종속항'})")
        print("📄 청구항 전체:", full_claim_text)
        print("📑 시작 문장 (raw):", meta["raw_text"][:200])


# 예시!
search_korpat("상,하부틀의 위치하는 단열재와, 상,하부틀과 단열재사이에 분할되게 삽입되어 단열재와 상,하부틀사이에 공간부를 형성하는 중간체와, 상기 단열제의 외측에 부착되는 판체로 구성된 적충체와", top_k=5)
print("\n\n")
search_korpat("본 발명은 자기 터널 접합(magnetic tunnel junction: MTJ)을 구성하기 위해 두 개의 고정층 사이에 하나의 자유층을 샌드위치처럼 순차적으로 수직하게 위치되는 자유층과 고정층 대비 쓰기 효율성을 향상시키는 스핀 주입 토크 자성메모리에 관한 것이다.", top_k=5)


🔹 Top 1 (score: 0.9995)
🆔 특허번호: 1019400024294 | 청구항: 1 (독립항)
📄 청구항 전체: 내부에 히터가 설치된 상,하부틀과상,하부틀의 위치하는 단열재와, 상,하부틀과 단열재사이에 분할되게 삽입되어 단열재와 상,하부틀사이에 공간부를 형성하는 중간체와, 상기 단열제의 외측에 부착되는 판체로 구성된 적충체와상기 적충체를 결착하기 위한 채결 수단과그 체결수단을 수용하는 적충체의 구멍과 채결수단사이에 열변형을 방지하기 위한 틈새가 형성된 것을 특징으로 하는 쉐도우 마스크 금형
📑 시작 문장 (raw): <ANY>상,하부틀의 위치하는 단열재와, 상,하부틀과 단열재사이에 분할되게 삽입되어 단열재와 상,하부틀사이에 공간부를 형성하는 중간체와, 상기 단열제의 외측</ANY><RELATION_396841>에 부착되는 판체로 구성된</RELATION_396841> <COMPONENT>적충체</COMPONENT>와

🔹 Top 2 (score: 0.9942)
🆔 특허번호: 1020070072388 | 청구항: 2 (종속항)
📄 청구항 전체: 청구항 1에 있어서,상기 상,하부 케이스의 마주하는 면에는 서로 대응되는 형상의 돌기와 홈이 각각 형성되어 상기 상,하부 케이스를 조립할 경우 정해진 위치로 안내되어 정확하게 조립될 수구성된것을특징으로 하는커넥터.
📑 시작 문장 (raw): <DIRECTION>청구항 1에 있어서,</DIRECTION><COMPONENT>상기 상</COMPONENT><ANY>,하부 케이스의 마주하는 면에는 서로 대응되는 형상의 돌기와 홈이 각각 형성되어 상기 상,하부 케이스를 조립할 경우 정해진 위치로 안내되어 정확하게 조립될 수</ANY><RELATION_015>구성된것을</RELATION_015><RELAT

🔹 Top 3 (score: 0.9937)
🆔 특허번호: 1020060064458 | 청구항: 1 (독립항)
📄 청구항 전체: 빛을 수광하여 전기신호로 변환하는 수광영역과, 상기 수광영역을 외부와 전기적으로 연결하

## 1/4조각

In [ ]:
import os
import json
import random
import numpy as np
import faiss
from collections import defaultdict
from tqdm import tqdm
import torch
import torch.nn.functional as F

# 설정
INDEX_PATH = "/content/drive/MyDrive/BIG_PROJ/korpat_index.faiss"
META_PATH = INDEX_PATH + ".meta.json"
OUT_INDEX_PATH = "/content/korpat_index_quarter_IP.faiss"
OUT_META_PATH = OUT_INDEX_PATH + ".meta.json"
TARGET_RATIO = 0.25  # 1/4

# 인덱스와 메타데이터 로드
print("📦 Loading index...")
index = faiss.read_index(INDEX_PATH)

print("📄 Loading metadata...")
with open(META_PATH, "r", encoding="utf-8") as f:
    metadata = json.load(f)

assert index.ntotal == len(metadata), "❗ 인덱스와 메타데이터 수 불일치"

# 그룹핑 (app_num + claim_num)
group_to_indices = defaultdict(list)
for idx, meta in enumerate(metadata):
    key = f"{meta['app_num']}_{meta['claim_num']}"
    group_to_indices[key].append(idx)

# 무작위 섞기
group_keys = list(group_to_indices.keys())
random.shuffle(group_keys)

# 목표 수만큼 그룹 선택
selected_indices = []
current_total = 0
target_total = int(len(metadata) * TARGET_RATIO)

for key in tqdm(group_keys):
    indices = group_to_indices[key]
    if current_total + len(indices) > target_total:
        continue
    selected_indices.extend(indices)
    current_total += len(indices)
    if current_total >= target_total:
        break

print(f"✅ 선택된 벡터 수: {current_total:,} / {len(metadata):,}")

# 선택된 벡터 추출
print("🎯 선택된 벡터 추출 중...")
dim = index.d
new_index = faiss.IndexFlatIP(dim)

# reconstruct 후 정규화
selected_vectors = np.stack([index.reconstruct(i) for i in selected_indices])
selected_vectors = F.normalize(torch.tensor(selected_vectors), p=2, dim=1).numpy()
new_index.add(selected_vectors.astype("float32"))

# 새 메타데이터 추출
selected_metadata = [metadata[i] for i in selected_indices]

# 저장
print("💾 Saving...")
faiss.write_index(new_index, OUT_INDEX_PATH)
with open(OUT_META_PATH, "w", encoding="utf-8") as f:
    json.dump(selected_metadata, f, ensure_ascii=False, indent=2)

print("🎉 Done.")


📦 Loading index...
📄 Loading metadata...


 25%|██▍       | 1299747/5205677 [00:02<00:06, 624102.78it/s]


✅ 선택된 벡터 수: 1,666,555 / 6,666,220
🎯 선택된 벡터 추출 중...
💾 Saving...
🎉 Done.


### 메타데이터 확인

In [ ]:
# Print the first 10 selected metadata entries with is_independent: True in a readable format
if selected_metadata:
    independent_metadata = [meta for meta in selected_metadata if meta.get('is_independent') is True]
    print(f"\n📋 선택된 독립항 메타데이터 미리보기 (상위 {min(10, len(independent_metadata))}개):")
    for i, meta in enumerate(independent_metadata[:10]):
        print(f"\n--- Entry {i+1} ---")
        for key, value in meta.items():
            # Limit output length for readability
            if isinstance(value, str) and len(value) > 200:
                print(f"{key}: {value[:200]}...")
            else:
                print(f"{key}: {value}")
else:
    print("\n⚠️ 선택된 메타데이터가 비어있습니다.")


📋 선택된 독립항 메타데이터 미리보기 (상위 10개):

--- Entry 1 ---
app_num: 2019950018228
claim_num: 1
is_independent: True
sent_num: 1
sent_total: 1
is_last: True
plain_text: 1. 메인샤시(12)의 일측면 상부로 부터는 상·하면의 중심에 스크류체결홈(14)이 형성된 체결보스(16)가 일정한 높이로 절곡 형성되고, 상기 체결보스(16)의 형성위치에 해당되는 메인베이스(18)의 면상에는 스크류관통공(20)이, 상기 체결보스(16)의 형성위치에 해당되는 하부커버(22)의 면상에는 중심에 스크류관통공(20)이 형성된 절곡부(24)가 윗...
raw_text: <ANY>1. 메인샤시(12)의 일측면 상부로 부터는 상·하면의 중심에 스크류체결홈(14)이 형성된 체결보스(16)가 일정한 높이로 절곡 형성되고, 상기 체결보스(16)의 형성위치에 해당되는 메인베이스(18)의 면상에는 스크류관통공(20)이, 상기 체결보스(16)의 형성위치에 해당되는 하부커버(22)의 면상에는 중심에 스크류관통공(20)이 형성된 절곡부(2...

--- Entry 2 ---
app_num: 1020150066943
claim_num: 1
is_independent: True
sent_num: 1
sent_total: 4
is_last: False
plain_text: 복수개의 프레임이 개구를 형성하는 프레임부
raw_text: <ANY>복수개의 프레임이 개구</ANY><RELATION_20>를 형성하는</RELATION_20> <COMPONENT>프레임부</COMPONENT>

--- Entry 3 ---
app_num: 1020150066943
claim_num: 1
is_independent: True
sent_num: 2
sent_total: 4
is_last: False
plain_text: 제1 방향으로 인장되어 상기 프레임부에 고정되는 증착용 마스크; 및
raw_text: <AN

### test

In [ ]:
from collections import defaultdict


def search_korpat(query, top_k=5, index=new_index, metadata=selected_metadata):
    with torch.inference_mode():
        inputs = tokenizer(query, return_tensors="pt", padding="max_length", truncation=True,
                           max_length=256, return_token_type_ids=False).to(DEVICE)
        vec = model(**inputs).last_hidden_state[:, 0, :]
        vec = torch.nn.functional.normalize(vec, p=2, dim=1).cpu().numpy()

    scores, indices = index.search(vec, top_k)

    for i, idx in enumerate(indices[0]):
        meta = metadata[idx]
        app_no = meta["app_num"]
        claim_no = meta["claim_num"]
        is_indep = meta["is_independent"]

        # 청구항에 속한 모든 문장 추출 및 정렬
        claim_sentences = sorted(
            [m for m in metadata if m["app_num"] == app_no and m["claim_num"] == claim_no],
            key=lambda x: x["sent_num"]
        )
        full_claim_text = "".join([m["plain_text"] for m in claim_sentences])

        print(f"\n🔹 Top {i+1} (score: {scores[0][i]:.4f})")
        print(f"🆔 특허번호: {app_no} | 청구항: {claim_no} ({'독립항' if is_indep else '종속항'})")
        print("📄 청구항 전체:", full_claim_text)
        print("📑 시작 문장 (raw):", meta['raw_text'][:200])

# 예시!
search_korpat("상,하부틀의 위치하는 단열재와, 상,하부틀과 단열재사이에 분할되게 삽입되어 단열재와 상,하부틀사이에 공간부를 형성하는 중간체와, 상기 단열제의 외측에 부착되는 판체로 구성된 적충체와", top_k=5)


🔹 Top 1 (score: 0.9938)
🆔 특허번호: 1019980060607 | 청구항: 1 (독립항)
📄 청구항 전체: 상부 내측면에 스테이터 코어가 마련되며, 중심면에는 관통구멍이 형성된 하우징과상기 하우징의 관통구멍에 하단부가 억지 끼워 삽입되는 고정축과하부 내측면에 요크와 마그네트가 마련되며, 중심면에는 가이드 구멍이 관통 형성되고, 이 가이드 구멍의 내주면과 고정축 사이에 소정간격으로 개재되는 상,하부 볼 베어링을 통해 고정축에 회전 가능하게 구름 베어링 지지되는 허브와상기 허브와 고정축 사이에 마련되어 상기 상,하부 볼 베어링을 지지하는 베어링 지지수단을 포함하여 된 것을 특징으로 하는 자기 디스크 구동모터
📑 시작 문장 (raw): <COMPONENT>하부 내측면에 요크와 마그네트가 마련되며, 중심면에는 가이드 구멍이 관통 형성되고, 이 가이드 구멍의 내주면과 고정축 사이에 소정간격으로 개재되는 상,하부 볼 베어링을 통해 고정축에 회전 가능하게 구름 베어링 지지되는 허브</COMPONENT>와

🔹 Top 2 (score: 0.9936)
🆔 특허번호: 1019870000924 | 청구항: 21 (독립항)
📄 청구항 전체: 전기응용기기를 제어하는 프로그램타이머에 있어서,하우징수단과, 하우징 수단상에 설치된 감속기 수단과,하우징수단에 설치되고 감속기 수단을 구동시키도록 접속된 타이밍모터와, 하우징수단상에서 회전 전진 하도록 설치된 주기간캠과,주기간캠의 적절한 시기의 전진을 하게하고, 래치휘일과 접촉하는 전진포올을 구비하면서, 전진포올이 래치휘일을 전진시키는 진동을 하도록 감속기 수단에 작동가능하게 접속되게한 전진수단과,하우징수단상에 저어널되는 표면과,그의 내측주변부 상에 형성된 톱니를 가진 링기어를 구비하며, 톱니가 저어널되는 표면상에 직접 저어널된 포인트로 되게한 프로그램타이머와, 링기어톱니와 결합하고 링기어를 구동시키는 감속기에 작동가능하게 접속된 피니언수단과,스위치수단과,부기간 캠수단에 응동하고 부기간캠수단의 전진에 응동하여 스위치수단을 

# 최종

In [ ]:
# ==== 셀 1: 준비/로드(한 번만 실행) ====
from typing import Dict
import os, json, re
import torch
import numpy as np
import faiss

from transformers import BertModel, BertTokenizer
from openai import OpenAI

# ===== 환경 설정 =====
# Colab 런타임 새로 시작하면 반드시 이 셀부터 실행
client = OpenAI(api_key=OPENAI_API_KEY)

# 경로 설정 (Drive 경로 그대로 사용 가능하지만 처음 한 번 /content로 복사하면 I/O 빠릅니다)
INDEX_PATH = "/content/drive/MyDrive/BIG_PROJ/korpat_index_quarter.faiss"
META_PATH  = INDEX_PATH + ".meta.json"

# 성능 최적화를 위해 (선택) 한 번 /content 로 복사
# - Drive I/O 느림 → 복사 후 /content/* 로 읽기 권장
# - 이미 /content 에 있으면 아래 두 줄은 생략 가능
!mkdir -p /content/korpat_index
!cp -u "{INDEX_PATH}" "/content/korpat_index/korpat_index.faiss"
!cp -u "{META_PATH}"  "/content/korpat_index/korpat_index.faiss.meta.json"

INDEX_PATH_LOCAL = "/content/korpat_index/korpat_index.faiss"
META_PATH_LOCAL  = "/content/korpat_index/korpat_index.faiss.meta.json"

# ===== 디바이스/torch 설정 =====
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cuda.matmul.allow_tf32 = True  # 가능하면 TF32 허용
torch.set_float32_matmul_precision("high")    # PyTorch 2.x

# ===== KorPat 임베딩 모델/토크나이저 로드 =====
MODEL_NAME = "KIPI-ai/KorPatElectra"

# Electra 계열을 BertModel로 로드하면 경고가 날 수 있습니다.
# 해당 체크포인트가 BERT호환 출력(클래스 토큰)만 쓰면 실사용 가능.
model = BertModel.from_pretrained(MODEL_NAME, token=ACCESS_TOKEN).to(DEVICE).eval()
if DEVICE.type == "cuda":
    model = model.half()

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME, token=ACCESS_TOKEN)

# ===== FAISS 인덱스/메타데이터 로드 =====
# (선택) 메모리맵으로 읽고 싶으면: faiss.read_index(INDEX_PATH_LOCAL, faiss.IO_FLAG_MMAP)
try:
    index = faiss.read_index(INDEX_PATH_LOCAL)
except TypeError:
    # 일부 환경은 IO_FLAG_MMAP 미지원일 수 있어 일반 로드로 대체
    index = faiss.read_index(INDEX_PATH_LOCAL)

with open(META_PATH_LOCAL, "r", encoding="utf-8") as f:
    metadata = json.load(f)

print("✅ 준비 완료")
print(f" - DEVICE: {DEVICE}")
print(f" - index.ntotal: {index.ntotal}")


config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/454M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/280k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

✅ 준비 완료
 - DEVICE: cuda
 - index.ntotal: 1666555


In [ ]:
# ==== 셀 2: 추론(요청마다 실행) ====
import re
import torch
import json

# ✅ 메타데이터 포함 RAG: (app_num, claim_num, text, score)
def retrieve_similar_claims_with_meta(query: str, model, tokenizer, index, metadata, device, top_k: int = 5):
    import numpy as np

    with torch.inference_mode():
        inputs = tokenizer(
            query,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=256,
            return_token_type_ids=False
        ).to(device)
        vec = model(**inputs).last_hidden_state[:, 0, :]
        vec = torch.nn.functional.normalize(vec, p=2, dim=1).cpu().numpy()

    scores, indices = index.search(vec, top_k)

    # (app_num, claim_num) 단위로 중복 제거 + 원문 구성
    seen = set()
    results = []
    for rank, idx in enumerate(indices[0]):
        meta = metadata[idx]
        pair = (meta["app_num"], meta["claim_num"])
        if pair in seen:
            continue
        seen.add(pair)

        claim_sents = [m for m in metadata if m["app_num"] == pair[0] and m["claim_num"] == pair[1]]
        claim_sents.sort(key=lambda x: x["sent_num"])
        claim_text = "".join(m["plain_text"] for m in claim_sents).strip()

        results.append({
            "rank": rank + 1,
            "score": float(scores[0][rank]) if isinstance(scores, np.ndarray) else None,
            "app_num": pair[0],
            "claim_num": pair[1],
            "text": claim_text
        })
    return results

# ✅ 호환 래퍼: 예전 코드가 기대한 "긴 문자열"을 반환
def retrieve_similar_claims(query: str, model, tokenizer, index, metadata, device, top_k: int = 5) -> str:
    meta = retrieve_similar_claims_with_meta(query, model, tokenizer, index, metadata, device, top_k)
    return "\n\n".join(item["text"] for item in meta)

def generate_patent_specification_from_claim_text(claim_text: str, rag_context) -> dict:
    # rag_context가 list(meta)면 텍스트로 변환, str이면 그대로 사용
    if isinstance(rag_context, list):
        rag_text = "\n\n".join(item.get("text","") for item in rag_context if isinstance(item, dict))
    else:
        rag_text = str(rag_context or "")

    def run_section_agent(title: str, instruction: str, context: str) -> str:
        common_instruction = (
            "각 항목은 '…이다/한다'의 현재형 문어체로 작성하고, 줄번호·글머리기호·표 목록을 사용하지 않는다. "
            "문장은 과장 표현 없이 객관적으로 기술하며, 동일 용어는 처음 정의한 표현을 일관되게 사용하되 각 항목에 중복되는 내용이 들어가지 않도록 주의한다. "
            "가능한 범위에서는 정량 표현(수치/범위/조건/기준선)을 포함하고, 원인→결과의 인과관계를 명시한다. "
            "추정·모호어(예: 대략, 상당히, 우수하다)는 금지하며, 항목 간 내용 중복을 피하고 서로 보완적으로 구성한다."
        )
        format_guard = (
            "출력 형식: 아래 JSON 스키마를 정확히 따르고, JSON 이외의 어떤 텍스트(설명/머리말/코드블록 표식)도 포함하지 말 것."
        )
        prompt = f"""
당신은 대한민국 특허 전문 변리사입니다.

{common_instruction}

{instruction}

{format_guard}

[입력 질의 또는 초안 요약]
{claim_text}

[유사 특허 청구항들]
{rag_text}

[이전 결과(참고 전용)]
{context}
""".strip()

        # 주의: client 객체는 상위 셀에서 생성되어 있어야 함 (예: OpenAI/PRELIM 설정)
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "당신은 대한민국 특허 전문 변리사입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
        )
        return resp.choices[0].message.content.strip()

    combined_text = f"""
[입력 질의 또는 초안 요약]
{claim_text}

[유사 특허 청구항들]
{rag_text}
""".strip()

    # ✅ 섹션 순서 업데이트: 요약은 맨 마지막, 청구항은 앞선 결과를 모두 참고하도록
    sections = [
        (
            "발명의 명칭",
            "단일 문자열로 작성한다. 기술 대상을 명확히 하며, 과장·광고성 표현과 불필요한 수식(차세대, 혁신적 등)은 금지한다. "
            "가능하면 핵심 수단 또는 구성요소를 포함하되, 30자 이내를 권장한다. 줄바꿈 없이 작성한다.\n"
            'JSON 스키마: {"발명의 명칭": "<텍스트>"}'
        ),
        (
            "기술 분야 및 배경 기술",
            "다음 두 항목을 반드시 해당 문구로 시작하고, 중간에 줄바꿈 없이 **각각 한 문단**으로 작성한다. "
            "1) '기술 분야:' 200자 이상으로 대상 기술의 분야/적용 대상을 정의한다. "
            "2) '배경 기술:' [유사 특허 청구항들]을 근거로 종래 방법과 한계(성능·효율·안전·비용·신뢰성·규제)를 400자 이상으로 기술한다. "
            "해결책 서술은 금지한다.\n"
            'JSON 스키마: {"기술 분야": "<문단 텍스트>", "배경 기술": "<문단 텍스트>"}'
        ),
        (
            "해결하려는 과제 및 해결 수단",
            "두 항목을 각각 최소 300자 이상, 줄바꿈 없이 **각각 한 문단**으로 작성한다. "
            "1) '해결하려는 과제:' 종래 기술의 결함/병목/리스크를 지표나 조건(오차율, 처리량, 전력, 온도 등)과 함께 명시한다. "
            "2) '과제의 해결 수단:' 핵심 메커니즘·구성요소·상호작용·동작 단계를 기술하고, 과제-수단 대응을 명확히 연결한다. 도면/실시예 번호는 언급 금지.\n"
            'JSON 스키마: {"해결하려는 과제": "<문단 텍스트>", "과제의 해결 수단": "<문단 텍스트>"}'
        ),
        (
            "발명의 효과",
            "단일 문단으로 400자 이상 작성한다(문단 번호 불필요). "
            "핵심 효과(성능·효율·신뢰성·내구·에너지·공정·호환성)와 발생 원인, 적용 조건/범위, 부가 효과(비용 절감, 유지보수, 통합성, 환경/안전)를 포함한다. "
            "가능하면 비교 기준선(종래 대비 %)이나 범위를 예시하되 과장 금지. 해결 수단 반복은 피하고 효과의 메커니즘과 사용 시나리오 중심으로 정리한다.\n"
            'JSON 스키마: {"발명의 효과": "<문단 텍스트>"}'
        ),
        (
            "청구항",
            "[이전 결과]의 용어·정의·관계를 그대로 따르고, 새로운 기술요소·용어를 도입하지 않는다. "
            "[청구항 1], [청구항 2] 형식으로 **가능한 한 많은** 청구항을 작성한다. "
            "독립항은 발명의 필수 구성요소와 이들 간의 관계를 포괄적으로 규정하되, 기능적 표현만으로 끝내지 말고 구조·수단·동작을 구체적으로 특정한다. "
            "종속항은 '제 X항에 있어서,'로 시작하며, 상위항의 구성에 추가적인 구성·조건·형태·범위·동작 순서·알고리즘 단계·연결 방식 등을 부가하여 보호범위를 세분화한다. "
            "효과만 단독으로 기재하는 청구항은 작성하지 않는다. "
            "각 청구항은 한 문장으로 끝맺고, 선택적 표현(바람직하다/예를 들면/가능하다 등)은 금지한다. "
            "상대적 표현(얇은/두꺼운/넓은 등)은 기준 또는 수치 범위를 함께 정의한다. 중복 없이 상호 보완적으로 구성한다.\n"
            'JSON 스키마: {"청구항": ["[청구항 1] ...", "[청구항 2] ...", "..."]}'
        ),
        (
            "요약",
            "[이전 결과]를 압축적으로 반영하고, 새로운 내용·용어를 추가하지 않는다. 단일 문단 380~450자로 작성한다. "
            "구성: (1) 기술분야 한줄 → (2) 해결하려는 과제(지표/조건 포함) → (3) 과제의 해결 수단(핵심 메커니즘·구성요소·동작) → "
            "(4) 발명의 효과(비교 기준선 또는 범위 포함). 동일 용어를 일관되게 사용한다.\n"
            'JSON 스키마: {"요약": "<문단 텍스트>"}'
        )
    ]

    spec_parts = {}
    context_so_far = combined_text

    for title, instruction in sections:
        try:
            section_out = run_section_agent(title, instruction, context_so_far)
        except Exception:
            # 실패 시 빈 JSON
            if title == "발명의 명칭":
                section_out = '{"발명의 명칭": ""}'
            elif title == "기술 분야 및 배경 기술":
                section_out = '{"기술 분야": "", "배경 기술": ""}'
            elif title == "해결하려는 과제 및 해결 수단":
                section_out = '{"해결하려는 과제": "", "과제의 해결 수단": ""}'
            elif title == "발명의 효과":
                section_out = '{"발명의 효과": ""}'
            elif title == "청구항":
                section_out = '{"청구항": []}'
            else:  # 요약
                section_out = '{"요약": ""}'

        spec_parts[title] = section_out
        context_so_far += f"\n\n[{title} 결과]\n{section_out}"

    return spec_parts

def parse_specification(spec_parts: dict) -> dict:
    if not isinstance(spec_parts, dict):
        spec_parts = {}

    result = {
        "발명의 명칭": "",
        "요약": "",
        "기술 분야": "",
        "배경 기술": "",
        "해결하려는 과제": "",
        "과제의 해결 수단": "",
        "발명의 효과": "",
        "청구항": ""
    }

    def _safe_load(s: str) -> dict:
        if not isinstance(s, str):
            return {}
        try:
            return json.loads(s)
        except Exception:
            s2 = s.strip()
            if s2.startswith("```"):
                s2 = s2.strip("`")
            l, r = s2.find("{"), s2.rfind("}")
            if l != -1 and r != -1 and l < r:
                try:
                    return json.loads(s2[l:r+1])
                except Exception:
                    return {}
            return {}

    # 신규: 발명의 명칭
    d = _safe_load(spec_parts.get("발명의 명칭", ""))
    result["발명의 명칭"] = d.get("발명의 명칭", "").strip()

    # 기존: 기술 분야/배경 기술
    d = _safe_load(spec_parts.get("기술 분야 및 배경 기술", ""))
    result["기술 분야"] = d.get("기술 분야", "").strip()
    result["배경 기술"] = d.get("배경 기술", "").strip()

    # 기존: 과제/해결 수단
    d = _safe_load(spec_parts.get("해결하려는 과제 및 해결 수단", ""))
    result["해결하려는 과제"] = d.get("해결하려는 과제", "").strip()
    result["과제의 해결 수단"] = d.get("과제의 해결 수단", "").strip()

    # 기존: 발명의 효과
    d = _safe_load(spec_parts.get("발명의 효과", ""))
    result["발명의 효과"] = d.get("발명의 효과", "").strip()

    # 청구항
    d = _safe_load(spec_parts.get("청구항", ""))
    claims = d.get("청구항", [])
    if isinstance(claims, list):
        result["청구항"] = "\n\n".join([c.strip() for c in claims if isinstance(c, str) and c.strip()])

    # 요약(맨 마지막 생성) + 길이 후처리
    d = _safe_load(spec_parts.get("요약", ""))
    raw_summary = d.get("요약", "").strip()
    result["요약"] = _adjust_summary_length(raw_summary, min_len=380, max_len=450)

    return result

def _adjust_summary_length(text: str, min_len: int = 380, max_len: int = 450) -> str:
    """
    요약 길이를 400자 언저리로 맞추기 위한 간단한 후처리.
    - max_len을 초과하면 문장 경계 기준으로 최대한 자연스럽게 잘라줌.
    - min_len보다 짧다면 그대로 둔다(재생성은 모델 프롬프트에서 380~450자로 유도).
    """
    s = re.sub(r"\s+", " ", text).strip()
    n = len(s)
    if n <= max_len:
        return s
    # 문장 경계(마침표/물음표/느낌표/따옴표) 근처에서 컷
    cutoff = max_len
    # 뒤로 가며 가장 가까운 경계 탐색
    for i in range(max_len, max(max_len - 80, min_len) - 1, -1):
        if i < len(s) and s[i] in ".?!。！？”’\"'…":
            cutoff = i + 1
            break
    return s[:cutoff].rstrip()

# ===== 사용 예시: 여기에 쿼리만 바꿔서 실행 =====
# sample_query = """
# 본원 디자인 “물걸레 브러쉬 청소기”의 하부에는 바닥면을 쓸면서 동시에 닦을 수 있는 본체가 구비되고, 이 본체의 후방에는 밀대 아답터가 결합되어 손으로 밀어서 청소하기 위한 밀대가 회동되게 장착된 형상과 모양의 결합을 디자인의 창작내용의 요점으로 함.
# 1. 재질은 합성수지재, 폴리에스테르재, 금속재임.
# 2. 본원 디자인 “물걸레 브러쉬 청소기”는 먼지 및 미세이물질을 회전하며 흡입하는 회전브러쉬와 바닥면을 닦아주는 바닥걸레가 구비되어 쓸고 닦기를 동시에 수행 가능한 것임.
# """.strip()

# sample_query = """
# 본 발명은 수술용 로봇 암에 관한 것으로, 특히 수술 부위의 조직을 파지하거나 절단하는 기능을 정밀하고 안정적으로 수행할 수 있는 회동 구조를 갖춘 로봇 암에 관한 것이다. 본 발명에 따른 수술용 로봇 암은 수술 과정에서 사용되는 시술용 그립퍼와 이를 지지 및 회동시키는 메커니즘을 포함한다. 구체적으로, 본 발명은 시술 부위 조직의 파지 및 절단 중 적어도 하나의 기능을 수행하기 위한 시술용 그립퍼와, 상기 그립퍼를 하단에서 고정 지지하는 그립퍼 지지 디스크, 그리고 상기 지지 디스크의 중심 하단부에 연결된 중심바를 포함한다.
# 또한, 상기 중심바와 지지 디스크는 볼 조인트에 의해 연결되어, 지지 디스크가 X축 및 Y축 방향으로 자유롭게 회동될 수 있도록 구성된다. 상기 지지 디스크의 중심점을 기준으로 대칭 위치에 배치된 3지점의 둘레면에는 디스크 와이어가 연결되며, 이 디스크 와이어는 각각 하단으로 연장되어 별도의 모터 구동부와 연결된다. 모터 구동부는 상기 와이어를 상하 방향으로 구동함으로써 지지 디스크의 기울기 및 회동 각도를 정밀하게 제어할 수 있으며, 이를 통해 시술용 그립퍼가 원하는 방향으로 정밀하게 조작된다.
# 본 발명은 하나의 중심점을 기준으로 2축 회동이 이루어지는 단순하고 안정적인 구조를 채택함으로써, 복잡한 기계적 요소를 줄여 고장 가능성을 낮출 수 있다. 또한, 시술자가 직관적으로 조작할 수 있으며, 유지보수 또한 간편하다는 장점이 있다. 나아가, 와이어 구동 방식을 통해 미세한 제어가 가능하여 실제 수술 시 조직을 섬세하게 파지하거나 원하는 위치에서 정확하게 절단할 수 있어, 기존의 복잡한 로봇 암 구조에 비해 높은 정밀성과 안정성을 제공한다.
# 따라서, 본 발명에 따른 수술용 로봇 암은 정밀한 수술 환경에서 안정적이고 효율적인 조작을 가능하게 하며, 장치의 내구성과 경제성 면에서도 우수한 효과를 발휘할 수 있다.
# """.strip()


sample_query = """
초소형 고효율 BLDC 모터를 적용하고 팬의 날개와 하우징 전체를 중공형(中空形) 일체형 카본 복합소재로 설계하여, 내구성은 유지하면서도 무게를 획기적으로 줄인 선풍기. 이를 통해 휴대성을 극대화하고 다양한 거치 형태를 손쉽게 구현할 수 있도록 하여 사용 편의성을 높이는 기술.
""".strip()

# ▶︎ 예전 코드와 동일하게 사용해도 됨(문자열 RAG 반환)
rag_context = retrieve_similar_claims(
    query=sample_query,
    model=model,
    tokenizer=tokenizer,
    index=index,
    metadata=metadata,
    device=DEVICE,
    top_k=5
)

print("\n📚 [RAG 검색된 유사 청구항]\n")
if isinstance(rag_context, str):
    print(rag_context[:2000] + ("\n...\n" if len(rag_context) > 2000 else ""))
else:
    # 혹시 누가 메타 버전을 직접 넘겼다면 안전 출력
    preview = "\n\n".join(item.get("text","") for item in rag_context[:3])
    print(preview[:2000] + ("\n...\n" if len(preview) > 2000 else ""))

# 아래 주석을 해제해 실행하면 전체 스펙 생성/파싱까지 수행
spec_parts = generate_patent_specification_from_claim_text(sample_query, rag_context)
parsed = parse_specification(spec_parts)

print("\n🔹 발명의 명칭:", parsed.get("발명의 명칭",""))

print("\n🔹 요약\n")
print(parsed.get("요약",""))

for k in ["기술 분야","배경 기술","해결하려는 과제","과제의 해결 수단","발명의 효과"]:
    v = parsed.get(k,"")
    preview = v[:800] + ("\n...\n" if len(v) > 800 else "")
    print(f"\n🔹 {k}\n{preview}")

print("\n🔹 청구항\n")
print(parsed.get("청구항",""))



📚 [RAG 검색된 유사 청구항]

제1항 내지 제4항 중 어느 한 항에 있어서,가스가 상기 저장 어셈블리의 내부와 외부 사이를 통과할 수 있도록 상기 링(50)과 케이스(20) 사이 및/또는 링(50)과 덮개(40) 사이에 규정되어 있는적어도하나의가스제거통로를추가포함하는것을특징으로 하는전기 에너지 저장 어셈블리.

제 3항에 있어서,장치(200)는;수신상태 감지수단(250)의 감지상태 또는 상기 서브장치(300)로부터의 호출신호를 주기적으로 체크하여 배터리(281)의 전원 온/오프상태를 조절하도록 하는 절전기능을 갖는전원공급수단더포함하는것을특징으로 하는휴대폰의 무선 핸즈프리 시스템.

제 4항에 있어서,컨트롤러의 클럭 단자에 엔드 게이트를 연결시키고, 상기 엔드 게이트는 클럭 펄스신호와 충전기 연결 유무 신호를 그 입력으로 하여 휴면(sleep)상태나 서스펜드(suspend) 모드에서 led컨트롤러의전력소모를최소화시키는것을특징으로 하는led를 이용한 배터리 충전 상태 표시장치.

제1항에 있어서,쌍의 탄성형 실링부재(30)의 사이를 슬라이딩 이동하는 슬라이더 측면의 전, 후 끝단(221a)(221b)은 유선형으로 경사지게 형성되어, 밀착되는 탄성형 실링부재와의틈새형성억제하도록한것을특징으로 하는리니어 액추에이터의 실링장치.

제1항에 있어서,실내조절부(200)에는실내에서 조작할 수 있는 리모콘(18)을 제어기능을 구비하고, 제2송수신부(16)가 그 리모콘 신호를 직접 수신할 수 있도록 하여서 구성되는 수단을 더 포함하는 것을 특징으로 하는 전화선과 무선 인터페이스가능한실내온도조절장치.

🔹 발명의 명칭: 초소형 고효율 BLDC 모터를 적용한 중공형 카본 복합소재 선풍기

🔹 요약

본 발명은 초소형 고효율 BLDC 모터를 이용한 선풍기에 관한 기술로, 팬의 날개와 하우징을 중공형 일체형 카본 복합소재로 설계하여 무게를 약 30% 이상 감소시키면서도 내구성을 유지하는 데 중점을 둔다. 종래 기술의 선풍기는 금속 또는 플라스틱 재질로 제작되어 평균적으로 1.5k

In [ ]:
# ==== 셀 0: SAFE RESET (port 8000은 kill하지 않음) ====
import os, subprocess, signal, time
import requests

# 우리가 쓰는 자동 포트 후보 (서버 셀과 일치)
CANDIDATE_PORTS = [8010, 8080, 9000, 7000]
CLOUDFLARED_BIN = "/content/cloudflared"
NGINX_CONF = "/content/nginx.conf"
CF_LOG = "/content/cloudflared.log"

def run(cmd: str) -> str:
    return subprocess.run(
        cmd, shell=True, check=False,
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    ).stdout

def ports_from_cloudflared_log(log_path=CF_LOG):
    # 로그에 "http://127.0.0.1:PORT" 형태가 남아있으면 그 포트를 추가로 수집
    ports = set()
    try:
        if os.path.exists(log_path):
            txt = open(log_path, "r", encoding="utf-8", errors="ignore").read()
            import re
            for m in re.finditer(r"http://127\.0\.0\.1:(\d+)", txt):
                ports.add(int(m.group(1)))
    except Exception:
        pass
    return list(ports)

print("🧹 Targeted stop (only notebook-started processes) ...")

# 0) ngrok (있다면 종료)
out = run("ps -ef | grep -v grep | grep 'ngrok' | awk '{print $2}'")
for pid in [p for p in out.strip().splitlines() if p.isdigit()]:
    try:
        os.kill(int(pid), signal.SIGTERM)
        print(f" - ngrok killed: {pid}")
    except Exception as e:
        print(f" - ngrok skip {pid}: {e}")

# 1) cloudflared (/content/cloudflared로 띄운 것만)
out = run(f"ps -ef | grep -v grep | grep '{CLOUDFLARED_BIN}' | awk '{{print $2}}'")
for pid in [p for p in out.strip().splitlines() if p.isdigit()]:
    try:
        os.kill(int(pid), signal.SIGTERM)
        print(f" - cloudflared killed: {pid}")
    except Exception as e:
        print(f" - cloudflared skip {pid}: {e}")

# 2) nginx (우리 conf로 뜬 master만)
out = run(f"ps -ef | grep -v grep | grep 'nginx: master process nginx -c {NGINX_CONF}' | awk '{{print $2}}'")
for pid in [p for p in out.strip().splitlines() if p.isdigit()]:
    try:
        os.kill(int(pid), signal.SIGTERM)
        print(f" - nginx(master) killed: {pid}")
    except Exception as e:
        print(f" - nginx skip {pid}: {e}")

# 3) uvicorn: kill 금지! -> 내부 종료 엔드포인트로만 종료 시도
#    서버 코드에 해당 엔드포인트가 없으면 404/연결실패가 나와 자동 스킵됩니다.
ports_to_try = list(dict.fromkeys(CANDIDATE_PORTS + ports_from_cloudflared_log()))
for port in ports_to_try:
    if port == 8000:
        continue
    url = f"http://127.0.0.1:{port}/__internal__/shutdown"
    try:
        r = requests.post(url, timeout=1.5)
        if r.ok:
            print(f" - asked uvicorn on :{port} to shutdown (HTTP {r.status_code})")
            time.sleep(0.7)
    except Exception:
        # 해당 포트에 우리 서버가 없거나, 엔드포인트가 없으면 무시
        pass

print("🗑️ Removing temp files...")
for p in [NGINX_CONF, CF_LOG]:
    try:
        if os.path.exists(p):
            os.remove(p)
            print(f" - removed {p}")
    except Exception as e:
        print(f" - skip {p}: {e}")

# cloudflared 바이너리는 보존하려면 아래를 주석 처리
try:
    if os.path.exists(CLOUDFLARED_BIN):
        os.remove(CLOUDFLARED_BIN)
        print(f" - removed {CLOUDFLARED_BIN}")
except Exception as e:
    print(f" - skip {CLOUDFLARED_BIN}: {e}")

print("✅ SAFE reset done. 모델/인덱스는 메모리에 그대로 남아있습니다.")


🧹 Targeted stop (only notebook-started processes) ...
 - cloudflared killed: 50630
INFO:     127.0.0.1:52364 - "POST /__internal__/shutdown HTTP/1.1" 404 Not Found
🗑️ Removing temp files...
 - removed /content/cloudflared.log
 - removed /content/cloudflared
✅ SAFE reset done. 모델/인덱스는 메모리에 그대로 남아있습니다.


In [ ]:
!fuser -k 9129/tcp

In [ ]:
# ==== FastAPI (Colab용: 포트 9129 고정 + 헬스체크 + 선택적 Cloudflared) ====
# 전제: model, tokenizer, index, metadata, DEVICE, client,
#       generate_patent_specification_from_claim_text / parse_specification 가 이미 메모리에 있음.

!pip -q install fastapi "uvicorn[standard]" pydantic==2.* openai==1.* > /dev/null

import os, threading, time, subprocess, json, socket
from typing import Optional, Dict, Literal
from fastapi import FastAPI, Query
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field
import requests
import uvicorn

# ---------- 고정 포트 ----------
SERVER_HOST = "0.0.0.0"
SERVER_PORT = 9129  # << 고정

# (안전) 시작 전 해당 포트 점유 프로세스 종료
os.system(f"fuser -k {SERVER_PORT}/tcp >/dev/null 2>&1 || true")

# ---------- 헬스체크 ----------
def wait_for_health(port: int, timeout_s: float = 12.0) -> bool:
    url = f"http://127.0.0.1:{port}/health"
    t0 = time.time()
    while time.time() - t0 < timeout_s:
        try:
            r = requests.get(url, timeout=0.7)
            if r.ok and r.json().get("status") == "ok":
                return True
        except Exception:
            pass
        time.sleep(0.25)
    return False

# ---------- RAG (메타 포함 버전) ----------
def retrieve_similar_claims_with_meta(query: str, model, tokenizer, index, metadata, device, top_k: int = 5):
    import torch, numpy as np
    with torch.inference_mode():
        inputs = tokenizer(
            query,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=256,
            return_token_type_ids=False
        ).to(device)
        vec = model(**inputs).last_hidden_state[:, 0, :]
        vec = torch.nn.functional.normalize(vec, p=2, dim=1).cpu().numpy()

    scores, indices = index.search(vec, top_k)

    seen = set()
    results = []
    for rank, idx in enumerate(indices[0]):
        meta = metadata[idx]
        pair = (meta["app_num"], meta["claim_num"])
        if pair in seen:
            continue
        seen.add(pair)

        # 같은 청구항 전체 문장 합치기
        claim_sents = [m for m in metadata if m["app_num"] == pair[0] and m["claim_num"] == pair[1]]
        claim_sents.sort(key=lambda x: x["sent_num"])
        claim_text = "".join(m["plain_text"] for m in claim_sents).strip()

        results.append({
            "rank": rank + 1,
            "score": float(scores[0][rank]) if hasattr(scores, "__getitem__") else None,
            "app_num": pair[0],
            "claim_num": pair[1],
            "text": claim_text
        })
    return results

# ---------- FastAPI 앱 ----------
app = FastAPI(title="KorPat Spec API", version="1.3.0")
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], allow_credentials=True, allow_methods=["*"], allow_headers=["*"],
)

class GenerateRequest(BaseModel):
    query: str = Field(..., description="명세서 생성을 위한 입력 텍스트(요약/초안/요구사항 등)")
    top_k: Optional[int] = Field(5, ge=1, le=50, description="RAG 검색 개수")

@app.get("/health")
def health():
    return {"status": "ok"}

# 내부 종료 엔드포인트 (SAFE RESET에서 사용)
server_stop_flag = {"exit": False}
@app.post("/__internal__/shutdown")
def shutdown():
    server_stop_flag["exit"] = True
    return {"ok": True}

from fastapi import HTTPException
import traceback, sys

@app.post("/generate")
def generate(
    req: GenerateRequest,
    minimal: bool = Query(True, description="True면 sections_parsed만 반환"),
    include_raw: bool = Query(False, description="sections_raw 포함 여부"),
    include_rag_meta: bool = Query(True, description="RAG 메타데이터 포함 여부"),
    rag_format: Literal["meta","text"] = Query("meta", description="RAG 포맷(meta/text)")
) -> Dict:
    # 0) 필수 객체 사전 점검
    missing = []
    for name in ["model","tokenizer","index","metadata","DEVICE","client",
                 "retrieve_similar_claims_with_meta",
                 "generate_patent_specification_from_claim_text",
                 "parse_specification"]:
        if name not in globals():
            missing.append(name)
    if missing:
        # 초기화 누락: 바로 알려주기
        raise HTTPException(status_code=500, detail={
            "error": "Server not initialized",
            "missing": missing,
            "hint": "셀 1/2에서 모델, 인덱스, 함수들을 먼저 로드하세요."
        })

    try:
        # 1) RAG (메타)
        rag_meta = retrieve_similar_claims_with_meta(
            query=req.query,
            model=model, tokenizer=tokenizer, index=index, metadata=metadata, device=DEVICE,
            top_k=req.top_k or 5
        )
        rag_text = "\n\n".join([item["text"] for item in rag_meta])

        # 2) 생성
        spec_parts = generate_patent_specification_from_claim_text(req.query, rag_text)

        # 3) 파싱
        parsed = parse_specification(spec_parts)

        # 4) 응답
        resp = {"sections_parsed": parsed}
        if not minimal and include_raw:
            resp["sections_raw"] = spec_parts
        if include_rag_meta:
            resp["rag_context"] = rag_meta if rag_format == "meta" else rag_text
        return resp

    except Exception as e:
        # 스택을 JSON으로 돌려줘서 바로 원인 확인
        tb = traceback.format_exc()
        raise HTTPException(status_code=500, detail={
            "error": str(e),
            "type": e.__class__.__name__,
            "traceback": tb.splitlines()[-20:],  # 마지막 20줄만
        })

# ---------- Uvicorn 실행 (9129 고정) ----------
def _exit_watcher():
    import os, signal
    while True:
        if server_stop_flag.get("exit"):
            os.kill(os.getpid(), signal.SIGINT)
            break
        time.sleep(0.5)

def start_uvicorn():
    uvicorn.run(app, host=SERVER_HOST, port=SERVER_PORT, log_level="info")

th = threading.Thread(target=start_uvicorn, daemon=True)
th.start()
watcher = threading.Thread(target=_exit_watcher, daemon=True)
watcher.start()

if wait_for_health(SERVER_PORT):
    print(f"✅ Uvicorn healthy: http://127.0.0.1:{SERVER_PORT}/health")
else:
    print(f"⚠️ Health check failed on port {SERVER_PORT}")


@app.get("/debug/deps")
def debug_deps():
    deps = {}
    for name in ["model","tokenizer","index","metadata","DEVICE","client",
                 "retrieve_similar_claims_with_meta",
                 "generate_patent_specification_from_claim_text",
                 "parse_specification"]:
        deps[name] = name in globals()
    return {"ok": all(deps.values()), "deps": deps}


# ---------- (선택) Cloudflared 터널 ----------
USE_CLOUDFLARED = True

def ensure_cloudflared(bin_path="/content/cloudflared"):
    if os.path.exists(bin_path):
        return bin_path
    url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64"
    print("⬇️  Downloading cloudflared...")
    subprocess.run(["wget", "-q", "-O", bin_path, url], check=True)
    os.chmod(bin_path, 0o755)
    return bin_path

def start_cloudflared(port: int, bin_path="/content/cloudflared"):
    # 헬스 통과 확인 후 시작
    if not wait_for_health(port, timeout_s=3.0):
        print(f"❌ Health check failed on :{port}. Not starting Cloudflared.")
        return None
    # 기존 터널 종료
    subprocess.run(["pkill", "-f", "cloudflared"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    time.sleep(0.3)
    # URL 추출
    log_path = "/content/cloudflared.log"
    with open(log_path, "w") as lf:
        _ = subprocess.Popen([bin_path, "tunnel", "--url", f"http://127.0.0.1:{port}"], stdout=lf, stderr=lf)
    public_url = None
    for _ in range(60):
        time.sleep(0.5)
        if os.path.exists(log_path):
            txt = open(log_path, "r").read()
            for line in txt.splitlines():
                if "trycloudflare.com" in line:
                    for token in line.split():
                        if token.startswith("https://") and "trycloudflare.com" in token:
                            public_url = token.strip()
                            break
                if public_url:
                    break
            if public_url:
                break
    return public_url

public_url = None
if USE_CLOUDFLARED and wait_for_health(SERVER_PORT, timeout_s=2.0):
    try:
        bin_path = ensure_cloudflared()
        public_url = start_cloudflared(SERVER_PORT, bin_path)
        if public_url:
            print(f"🌍 Public URL (Cloudflared): {public_url}")
            print(f"   ➤ Health: {public_url}/health")
            print(f"   ➤ POST   {public_url}/generate")
            print("\n💡 Example usage (parsed-only):")
            print(f"""curl -s "{public_url}/generate?minimal=true&include_rag_meta=true&rag_format=meta" \\
  -H "Content-Type: application/json; charset=utf-8" \\
  --data-raw '{{"query":"자율주행 차량의 객체 인식 취약점 보완 장치 및 방법","top_k":5}}'""")
            print("\n💡 Example usage (Windows PowerShell, pretty):")
            print(f"""$body = @{{ query = "자율주행 차량의 객체 인식 취약점 보완 장치 및 방법"; top_k = 5 }} | ConvertTo-Json -Compress
Invoke-RestMethod -Uri "{public_url}/generate?minimal=true&include_rag_meta=true&rag_format=meta" -Method POST -ContentType "application/json; charset=utf-8" -Body $body | ConvertTo-Json -Depth 8""")
        else:
            print("⚠️ Cloudflared URL을 찾지 못했습니다. /content/cloudflared.log 를 확인하세요.")
    except Exception as e:
        print(f"❌ Cloudflared 실패: {e}")

print(f"📍 Local test: http://127.0.0.1:{SERVER_PORT}/health")

INFO:     Started server process [19286]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9129 (Press CTRL+C to quit)


INFO:     127.0.0.1:39240 - "GET /health HTTP/1.1" 200 OK
✅ Uvicorn healthy: http://127.0.0.1:9129/health
INFO:     127.0.0.1:39248 - "GET /health HTTP/1.1" 200 OK
INFO:     127.0.0.1:39264 - "GET /health HTTP/1.1" 200 OK
🌍 Public URL (Cloudflared): https://neil-gordon-georgia-thumbnail.trycloudflare.com
   ➤ Health: https://neil-gordon-georgia-thumbnail.trycloudflare.com/health
   ➤ POST   https://neil-gordon-georgia-thumbnail.trycloudflare.com/generate

💡 Example usage (parsed-only):
curl -s "https://neil-gordon-georgia-thumbnail.trycloudflare.com/generate?minimal=true&include_rag_meta=true&rag_format=meta" \
  -H "Content-Type: application/json; charset=utf-8" \
  --data-raw '{"query":"자율주행 차량의 객체 인식 취약점 보완 장치 및 방법","top_k":5}'

💡 Example usage (Windows PowerShell, pretty):
$body = @{ query = "자율주행 차량의 객체 인식 취약점 보완 장치 및 방법"; top_k = 5 } | ConvertTo-Json -Compress
Invoke-RestMethod -Uri "https://neil-gordon-georgia-thumbnail.trycloudflare.com/generate?minimal=true&include_rag_meta=tr